In [7]:
import os
import copy
import random
import time

import graphviz
from tqdm import tqdm
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

import optuna
from optuna.samplers import TPESampler

from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import warnings
warnings.filterwarnings('ignore')

seed = 42
random.seed(seed)
np.random.seed(seed)
sampler = TPESampler(seed=seed)

In [8]:
# Прочитаем датасет
dataset = pd.read_csv(
    '/home/rito4ka/dev/magistratura/miptml_projects/songs_classification/spotify-2023.csv',
    encoding='latin-1'
)

# Добавим категориальные коды для признаков
dataset['key'] = pd.Categorical(dataset['key'])
dataset['key'] = dataset['key'].cat.codes
dataset['mode'] = pd.Categorical(dataset['mode'])
dataset['mode'] = dataset['mode'].cat.codes

# Скастуем к инту то, что имело тип строки
dataset['in_deezer_playlists'] = [int(item.replace(',', '')) for item in dataset['in_deezer_playlists']]
dataset['in_shazam_charts'] = [int(item.replace(',', '')) if isinstance(item, str) else item
                               for item in dataset['in_shazam_charts']]

# Нагенерим столбец с целевым признаком
dataset['is_in_spotify_charts'] = [int(bool(item)) for item in dataset['in_spotify_charts']]

# Уберём лишние столбцы (Название трека и имя артиста; присутствие в плейлистах и чартах
# на платформах кроме Spotify, на Spotify скипнем только чарты, тк это целевой признак)
dataset = dataset.drop(
    ['artist(s)_name', 'in_spotify_charts', 'in_apple_playlists', 'in_apple_charts',
     'in_deezer_playlists', 'in_deezer_charts', 'in_shazam_charts'], axis=1
)

# Уберём признаки, которые решающее дерево нашло ненужными:
dataset = dataset.drop(
    ['artist_count', 'danceability_%', 'instrumentalness_%', 'mode', 'released_month', 'speechiness_%'], axis=1
)

dataset.head()

,track_name,released_year,released_day,in_spotify_playlists,streams,bpm,key,valence_%,energy_%,acousticness_%,liveness_%,is_in_spotify_charts
0,Seven (feat. Latto) (Explicit Ver.),2023,14,553,141381703.0,125,2,89,83,31,8,1
1,LALA,2023,23,1474,133716286.0,92,3,61,74,7,10,1
2,vampire,2023,30,1397,140003974.0,138,7,32,53,17,31,1
3,Cruel Summer,2019,23,7858,800840817.0,170,0,58,72,11,11,1
4,WHERE SHE GOES,2023,18,3133,303236322.0,144,0,23,80,14,11,1


In [9]:
# Уберём имя песни и разделим на train и test выборки.
dataset = dataset.drop('track_name', axis=1)

# Обработаем nan
dataset = dataset.fillna(0)

X = dataset.drop('is_in_spotify_charts', axis=1)
y = dataset['is_in_spotify_charts']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

X_train_original = copy.deepcopy(X_train)
X_test_original = copy.deepcopy(X_test)

# Произведём масштабирование:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

"Пооптуним" LogReg

In [55]:
def objective(trial):    
    C = trial.suggest_float('C', 0.0001, 1.0)
    
    model = LogisticRegression(
            penalty='l2',
            class_weight=None,
            C=C,
            solver='saga',
            max_iter=42,
            random_state=seed
        )
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

study = optuna.create_study(direction="maximize", sampler=sampler) 
study.optimize(objective, show_progress_bar=True, n_trials=5000)

[I 2023-11-15 12:20:26,453] A new study created in memory with name: no-name-8381ef55-eb4d-40b3-b1fa-47ff896e474e


  0%|          | 0/5000 [00:00<?, ?it/s]

[I 2023-11-15 12:20:26,482] Trial 0 finished with value: 0.5968586387434555 and parameters: {'C': 0.962451050212617}. Best is trial 0 with value: 0.5968586387434555.
[I 2023-11-15 12:20:26,490] Trial 1 finished with value: 0.5916230366492147 and parameters: {'C': 0.2518571175957816}. Best is trial 0 with value: 0.5968586387434555.
[I 2023-11-15 12:20:26,499] Trial 2 finished with value: 0.5968586387434555 and parameters: {'C': 0.4972987810417962}. Best is trial 0 with value: 0.5968586387434555.
[I 2023-11-15 12:20:26,506] Trial 3 finished with value: 0.5916230366492147 and parameters: {'C': 0.30094822198578797}. Best is trial 0 with value: 0.5968586387434555.
[I 2023-11-15 12:20:26,511] Trial 4 finished with value: 0.5916230366492147 and parameters: {'C': 0.2849120103280299}. Best is trial 0 with value: 0.5968586387434555.
[I 2023-11-15 12:20:26,515] Trial 5 finished with value: 0.6073298429319371 and parameters: {'C': 0.03698325865979735}. Best is trial 5 with value: 0.607329842931937

[I 2023-11-15 12:20:27,184] Trial 53 finished with value: 0.6335078534031413 and parameters: {'C': 0.0011842293597689843}. Best is trial 24 with value: 0.6492146596858639.
[I 2023-11-15 12:20:27,197] Trial 54 finished with value: 0.5968586387434555 and parameters: {'C': 0.12911017410319367}. Best is trial 24 with value: 0.6492146596858639.
[I 2023-11-15 12:20:27,208] Trial 55 finished with value: 0.6335078534031413 and parameters: {'C': 0.0018374671524312518}. Best is trial 24 with value: 0.6492146596858639.
[I 2023-11-15 12:20:27,219] Trial 56 finished with value: 0.5916230366492147 and parameters: {'C': 0.1562642359021552}. Best is trial 24 with value: 0.6492146596858639.
[I 2023-11-15 12:20:27,232] Trial 57 finished with value: 0.6073298429319371 and parameters: {'C': 0.06132302508601166}. Best is trial 24 with value: 0.6492146596858639.
[I 2023-11-15 12:20:27,239] Trial 58 finished with value: 0.5968586387434555 and parameters: {'C': 0.10824020631706478}. Best is trial 24 with valu

[I 2023-11-15 12:20:27,597] Trial 107 finished with value: 0.6073298429319371 and parameters: {'C': 0.043213715665437066}. Best is trial 24 with value: 0.6492146596858639.
[I 2023-11-15 12:20:27,604] Trial 108 finished with value: 0.6020942408376964 and parameters: {'C': 0.06958536891762836}. Best is trial 24 with value: 0.6492146596858639.
[I 2023-11-15 12:20:27,613] Trial 109 finished with value: 0.6335078534031413 and parameters: {'C': 0.0013795160211917653}. Best is trial 24 with value: 0.6492146596858639.
[I 2023-11-15 12:20:27,622] Trial 110 finished with value: 0.5968586387434555 and parameters: {'C': 0.12627649627010093}. Best is trial 24 with value: 0.6492146596858639.
[I 2023-11-15 12:20:27,631] Trial 111 finished with value: 0.6230366492146597 and parameters: {'C': 0.013415029209008614}. Best is trial 24 with value: 0.6492146596858639.
[I 2023-11-15 12:20:27,638] Trial 112 finished with value: 0.6073298429319371 and parameters: {'C': 0.03477019328851166}. Best is trial 24 wi

...

[I 2023-11-15 12:21:52,695] Trial 4754 finished with value: 0.6178010471204188 and parameters: {'C': 0.016512955358938364}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:52,721] Trial 4755 finished with value: 0.5968586387434555 and parameters: {'C': 0.5857243800956575}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:52,749] Trial 4756 finished with value: 0.6073298429319371 and parameters: {'C': 0.03465141506714936}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:52,782] Trial 4757 finished with value: 0.6230366492146597 and parameters: {'C': 0.0003983614962594674}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:52,822] Trial 4758 finished with value: 0.5916230366492147 and parameters: {'C': 0.3634275429010741}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:52,851] Trial 4759 finished with value: 0.5968586387434555 and parameters: {'C': 0.9530829752589838}. Best is tria

[I 2023-11-15 12:21:54,235] Trial 4805 finished with value: 0.6073298429319371 and parameters: {'C': 0.03327768632885557}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:54,274] Trial 4806 finished with value: 0.5968586387434555 and parameters: {'C': 0.1280441311130291}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:54,300] Trial 4807 finished with value: 0.6020942408376964 and parameters: {'C': 0.06613987561229036}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:54,334] Trial 4808 finished with value: 0.5968586387434555 and parameters: {'C': 0.8750770094660312}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:54,360] Trial 4809 finished with value: 0.612565445026178 and parameters: {'C': 0.018128277891371863}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:54,394] Trial 4810 finished with value: 0.5916230366492147 and parameters: {'C': 0.44394222995492844}. Best is trial 

[I 2023-11-15 12:21:55,744] Trial 4856 finished with value: 0.5916230366492147 and parameters: {'C': 0.29570938659610185}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:55,775] Trial 4857 finished with value: 0.5916230366492147 and parameters: {'C': 0.1634748533351866}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:55,805] Trial 4858 finished with value: 0.6020942408376964 and parameters: {'C': 0.020536925539695987}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:55,831] Trial 4859 finished with value: 0.5916230366492147 and parameters: {'C': 0.36578779114386156}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:55,859] Trial 4860 finished with value: 0.6178010471204188 and parameters: {'C': 0.01691709996223129}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:55,893] Trial 4861 finished with value: 0.5916230366492147 and parameters: {'C': 0.4128980332394773}. Best is trial

[I 2023-11-15 12:21:57,245] Trial 4907 finished with value: 0.5916230366492147 and parameters: {'C': 0.20200558564358417}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:57,272] Trial 4908 finished with value: 0.6073298429319371 and parameters: {'C': 0.03405556407068582}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:57,302] Trial 4909 finished with value: 0.6073298429319371 and parameters: {'C': 0.018298259654622416}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:57,337] Trial 4910 finished with value: 0.5968586387434555 and parameters: {'C': 0.5222722238191911}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:57,363] Trial 4911 finished with value: 0.5968586387434555 and parameters: {'C': 0.9452002000832973}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:57,391] Trial 4912 finished with value: 0.612565445026178 and parameters: {'C': 0.06084356423737097}. Best is trial 

[I 2023-11-15 12:21:58,814] Trial 4958 finished with value: 0.5968586387434555 and parameters: {'C': 0.6362950709609795}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:58,839] Trial 4959 finished with value: 0.5968586387434555 and parameters: {'C': 0.9097356975131554}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:58,863] Trial 4960 finished with value: 0.6020942408376964 and parameters: {'C': 0.04712067206321911}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:58,889] Trial 4961 finished with value: 0.6178010471204188 and parameters: {'C': 0.016757172279345766}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:58,919] Trial 4962 finished with value: 0.5968586387434555 and parameters: {'C': 0.49878328997067545}. Best is trial 135 with value: 0.6544502617801047.
[I 2023-11-15 12:21:58,953] Trial 4963 finished with value: 0.5916230366492147 and parameters: {'C': 0.8312942499337843}. Best is trial 

In [57]:
study.best_trial

FrozenTrial(number=135, state=1, values=[0.6544502617801047], datetime_start=datetime.datetime(2023, 11, 15, 12, 20, 27, 866888), datetime_complete=datetime.datetime(2023, 11, 15, 12, 20, 27, 874325), params={'C': 0.004184237688072954}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'C': FloatDistribution(high=1.0, log=False, low=0.0001, step=None)}, trial_id=135, value=None)

In [17]:
model = LogisticRegression(
            penalty='l2',
            class_weight=None,
            C=0.007646564584361304,
            solver='saga',
            max_iter=42,
            random_state=seed
        )
model.fit(X_train, y_train)
        
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.3f}')

f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1:.3f}')

report = classification_report(y_test, y_pred)
print('Classification Report:\n', report)

Accuracy: 0.654
F1 score: 0.761
Classification Report:
               precision    recall  f1-score   support

           0       0.59      0.28      0.38        72
           1       0.67      0.88      0.76       119

    accuracy                           0.65       191
   macro avg       0.63      0.58      0.57       191
weighted avg       0.64      0.65      0.62       191



А теперь SVC

In [53]:
def objective(trial):    
    C = trial.suggest_float('C', 0.0001, 1.0)
    coef0 = trial.suggest_float('coef0', 0.0001, 1.0)
    max_iter = trial.suggest_int('max_iter', 1, 125)
#     degree = trial.suggest_int('degree', 1, 5)
    
    model = SVC(
        class_weight=None,
        C=C,
        degree=2,
        gamma='scale',
        coef0=coef0,
        probability=False,
        kernel='poly',
        max_iter=max_iter,
        random_state=seed
    )
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

study = optuna.create_study(direction="maximize", sampler=sampler) 
study.optimize(objective, show_progress_bar=True, n_trials=5000)

[I 2023-11-15 12:15:44,426] A new study created in memory with name: no-name-24913ba9-add1-4adc-bbd9-f37173312e5f


  0%|          | 0/5000 [00:00<?, ?it/s]

[I 2023-11-15 12:15:44,442] Trial 0 finished with value: 0.6282722513089005 and parameters: {'C': 0.28982247776847664, 'coef0': 0.161305165125279, 'max_iter': 117}. Best is trial 0 with value: 0.6282722513089005.
[I 2023-11-15 12:15:44,449] Trial 1 finished with value: 0.6492146596858639 and parameters: {'C': 0.8081395675264605, 'coef0': 0.6334404161347724, 'max_iter': 109}. Best is trial 1 with value: 0.6492146596858639.
[I 2023-11-15 12:15:44,457] Trial 2 finished with value: 0.6020942408376964 and parameters: {'C': 0.8036917096914246, 'coef0': 0.18665140188014723, 'max_iter': 112}. Best is trial 1 with value: 0.6492146596858639.
[I 2023-11-15 12:15:44,464] Trial 3 finished with value: 0.6335078534031413 and parameters: {'C': 0.5393883076914592, 'coef0': 0.8074594111485461, 'max_iter': 113}. Best is trial 1 with value: 0.6492146596858639.
[I 2023-11-15 12:15:44,468] Trial 4 finished with value: 0.612565445026178 and parameters: {'C': 0.3180716746243667, 'coef0': 0.11014091933522399, 

[I 2023-11-15 12:15:45,053] Trial 46 finished with value: 0.612565445026178 and parameters: {'C': 0.6176836204876609, 'coef0': 0.8756256291001029, 'max_iter': 93}. Best is trial 33 with value: 0.6544502617801047.
[I 2023-11-15 12:15:45,068] Trial 47 finished with value: 0.612565445026178 and parameters: {'C': 0.8066436711363422, 'coef0': 0.9966065813963589, 'max_iter': 85}. Best is trial 33 with value: 0.6544502617801047.
[I 2023-11-15 12:15:45,083] Trial 48 finished with value: 0.6178010471204188 and parameters: {'C': 0.9143548067605473, 'coef0': 0.8164433004251073, 'max_iter': 106}. Best is trial 33 with value: 0.6544502617801047.
[I 2023-11-15 12:15:45,097] Trial 49 finished with value: 0.6178010471204188 and parameters: {'C': 0.5706825445122525, 'coef0': 0.9403572731761695, 'max_iter': 25}. Best is trial 33 with value: 0.6544502617801047.
[I 2023-11-15 12:15:45,115] Trial 50 finished with value: 0.6178010471204188 and parameters: {'C': 0.7124960973564907, 'coef0': 0.635302248019085

[I 2023-11-15 12:15:45,914] Trial 96 finished with value: 0.6230366492146597 and parameters: {'C': 0.6308031039032088, 'coef0': 0.7910469930929517, 'max_iter': 91}. Best is trial 33 with value: 0.6544502617801047.
[I 2023-11-15 12:15:45,935] Trial 97 finished with value: 0.6230366492146597 and parameters: {'C': 0.6061546488817934, 'coef0': 0.876942081920364, 'max_iter': 107}. Best is trial 33 with value: 0.6544502617801047.
[I 2023-11-15 12:15:45,957] Trial 98 finished with value: 0.6178010471204188 and parameters: {'C': 0.5780343739494546, 'coef0': 0.7337779026998243, 'max_iter': 113}. Best is trial 33 with value: 0.6544502617801047.
[I 2023-11-15 12:15:45,974] Trial 99 finished with value: 0.6230366492146597 and parameters: {'C': 0.6876180116808583, 'coef0': 0.7579264572677668, 'max_iter': 95}. Best is trial 33 with value: 0.6544502617801047.
[I 2023-11-15 12:15:45,990] Trial 100 finished with value: 0.6178010471204188 and parameters: {'C': 0.7141103466574539, 'coef0': 0.628590013959

...

[I 2023-11-15 12:19:48,622] Trial 4824 finished with value: 0.6335078534031413 and parameters: {'C': 0.05430094840256028, 'coef0': 0.7159021469835268, 'max_iter': 89}. Best is trial 3462 with value: 0.6963350785340314.
[I 2023-11-15 12:19:48,698] Trial 4825 finished with value: 0.6230366492146597 and parameters: {'C': 0.10398314519373142, 'coef0': 0.8321420490347607, 'max_iter': 91}. Best is trial 3462 with value: 0.6963350785340314.
[I 2023-11-15 12:19:48,773] Trial 4826 finished with value: 0.643979057591623 and parameters: {'C': 0.7790859033576272, 'coef0': 0.7757355333866772, 'max_iter': 53}. Best is trial 3462 with value: 0.6963350785340314.
[I 2023-11-15 12:19:48,850] Trial 4827 finished with value: 0.6178010471204188 and parameters: {'C': 0.8603289678539485, 'coef0': 0.8093199702394014, 'max_iter': 104}. Best is trial 3462 with value: 0.6963350785340314.
[I 2023-11-15 12:19:48,923] Trial 4828 finished with value: 0.6178010471204188 and parameters: {'C': 0.9049887868935846, 'coef

[I 2023-11-15 12:19:51,692] Trial 4862 finished with value: 0.6335078534031413 and parameters: {'C': 0.2723853557032567, 'coef0': 0.5835601721556927, 'max_iter': 100}. Best is trial 3462 with value: 0.6963350785340314.
[I 2023-11-15 12:19:51,819] Trial 4863 finished with value: 0.6073298429319371 and parameters: {'C': 0.8307926808485087, 'coef0': 0.8427306415581222, 'max_iter': 91}. Best is trial 3462 with value: 0.6963350785340314.
[I 2023-11-15 12:19:51,946] Trial 4864 finished with value: 0.6387434554973822 and parameters: {'C': 0.9846013564540671, 'coef0': 0.7331881024550052, 'max_iter': 94}. Best is trial 3462 with value: 0.6963350785340314.
[I 2023-11-15 12:19:52,080] Trial 4865 finished with value: 0.6178010471204188 and parameters: {'C': 0.8756374983456153, 'coef0': 0.7938735059689728, 'max_iter': 84}. Best is trial 3462 with value: 0.6963350785340314.
[I 2023-11-15 12:19:52,215] Trial 4866 finished with value: 0.6178010471204188 and parameters: {'C': 0.4441243881843428, 'coef0

[I 2023-11-15 12:19:56,450] Trial 4901 finished with value: 0.6230366492146597 and parameters: {'C': 0.9995866878533908, 'coef0': 0.8401099961525121, 'max_iter': 102}. Best is trial 3462 with value: 0.6963350785340314.
[I 2023-11-15 12:19:56,562] Trial 4902 finished with value: 0.6335078534031413 and parameters: {'C': 0.3351263119992298, 'coef0': 0.924200759425817, 'max_iter': 88}. Best is trial 3462 with value: 0.6963350785340314.
[I 2023-11-15 12:19:56,675] Trial 4903 finished with value: 0.6335078534031413 and parameters: {'C': 0.7348043824506634, 'coef0': 0.7233527453531771, 'max_iter': 97}. Best is trial 3462 with value: 0.6963350785340314.
[I 2023-11-15 12:19:56,800] Trial 4904 finished with value: 0.6230366492146597 and parameters: {'C': 0.765515553900671, 'coef0': 0.22634392565797745, 'max_iter': 91}. Best is trial 3462 with value: 0.6963350785340314.
[I 2023-11-15 12:19:56,925] Trial 4905 finished with value: 0.6282722513089005 and parameters: {'C': 0.8446248533240102, 'coef0'

[I 2023-11-15 12:20:00,983] Trial 4939 finished with value: 0.6230366492146597 and parameters: {'C': 0.9167809540276697, 'coef0': 0.4163817831313516, 'max_iter': 93}. Best is trial 3462 with value: 0.6963350785340314.
[I 2023-11-15 12:20:01,086] Trial 4940 finished with value: 0.5916230366492147 and parameters: {'C': 0.8955270535975381, 'coef0': 0.8649028785652515, 'max_iter': 100}. Best is trial 3462 with value: 0.6963350785340314.
[I 2023-11-15 12:20:01,220] Trial 4941 finished with value: 0.6178010471204188 and parameters: {'C': 0.5878099085289916, 'coef0': 0.5211209920768354, 'max_iter': 50}. Best is trial 3462 with value: 0.6963350785340314.
[I 2023-11-15 12:20:01,320] Trial 4942 finished with value: 0.6178010471204188 and parameters: {'C': 0.504708909526935, 'coef0': 0.8144901376419788, 'max_iter': 96}. Best is trial 3462 with value: 0.6963350785340314.
[I 2023-11-15 12:20:01,446] Trial 4943 finished with value: 0.6178010471204188 and parameters: {'C': 0.9542291273976107, 'coef0'

[I 2023-11-15 12:20:05,704] Trial 4978 finished with value: 0.643979057591623 and parameters: {'C': 0.8570302886083024, 'coef0': 0.8134254574127427, 'max_iter': 91}. Best is trial 3462 with value: 0.6963350785340314.
[I 2023-11-15 12:20:05,804] Trial 4979 finished with value: 0.6335078534031413 and parameters: {'C': 0.8132278636350211, 'coef0': 0.7128041755946447, 'max_iter': 95}. Best is trial 3462 with value: 0.6963350785340314.
[I 2023-11-15 12:20:05,893] Trial 4980 finished with value: 0.6282722513089005 and parameters: {'C': 0.8850266508286797, 'coef0': 0.7738934537168782, 'max_iter': 92}. Best is trial 3462 with value: 0.6963350785340314.
[I 2023-11-15 12:20:06,011] Trial 4981 finished with value: 0.6282722513089005 and parameters: {'C': 0.16097324858155476, 'coef0': 0.8672170261748641, 'max_iter': 89}. Best is trial 3462 with value: 0.6963350785340314.
[I 2023-11-15 12:20:06,127] Trial 4982 finished with value: 0.6387434554973822 and parameters: {'C': 0.37688894601196626, 'coef0

In [54]:
study.best_value, study.best_params

(0.6963350785340314,
 {'C': 0.8693513056724096, 'coef0': 0.7919349229668778, 'max_iter': 96})

In [60]:
model = SVC(
    class_weight=None,
    C=0.8693513056724096,
    degree=2,
    gamma='scale',
    coef0=0.7919349229668778,
    probability=False,
    kernel='poly',
    max_iter=96,
    random_state=seed
)
model.fit(X_train, y_train)
    
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.3f}')

f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1:.3f}')

report = classification_report(y_test, y_pred)
print('Classification Report:\n', report)

Accuracy: 0.696
F1 score: 0.779
Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.43      0.52        72
           1       0.71      0.86      0.78       119

    accuracy                           0.70       191
   macro avg       0.68      0.64      0.65       191
weighted avg       0.69      0.70      0.68       191

